<a href="https://colab.research.google.com/github/sudhir2016/India_Shallow_Groundwater/blob/main/India_Shallow_Groundwater.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Geopandas and Rasterio

In [ ]:
!pip install geopandas

In [ ]:
!pip install rasterio

Import various libraries.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import fiona as fn
import rasterio as rio
from rasterio import mask

Open Global Shallow Groundwater TIFF file.

In [4]:
data=rio.open('/content/gws_perc_025deg_GL.tif')

Obtain CRS.

In [5]:
crs=data.crs
print(crs)

EPSG:4326


Extract India(partial) GeoJSON file from Geopandas natural earth dataset.

In [6]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [7]:
world1=world.set_index('name')

In [8]:
India=world1.loc['India','geometry']

In [9]:
India1=gpd.GeoSeries([India])

In [10]:
India1.to_file("India.geojson", driver='GeoJSON')

Read India(partial) GeoJSON file and use it to mask the Global Shallow Groundwater data.

In [12]:
India='/content/India.geojson'

In [13]:
map=gpd.read_file(India)

In [ ]:
map1=map.to_crs({'init': crs})

In [ ]:
geo=map1.geometry
print(geo)

In [ ]:
map.plot()

In [18]:
with fn.open(India,'r') as shapefile:
  shapes=[feature['geometry'] for feature in shapefile]

In [20]:
with rio.open("/content/gws_perc_025deg_GL.tif") as src:
    out_image, out_transform = rio.mask.mask(src,geo,crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

Obtain India Shallow Groundwater TIFF file.

In [21]:
with rio.open('SGW_India.tiff', 'w', **out_meta) as dest:
    dest.write(out_image)

Plot India Shallow Groundwater map.

In [ ]:
src = rio.open('/content/SGW_India.tiff')
src1=src.read(1,masked=True)
plt.imshow(src1,vmin=0,vmax=100,cmap='RdYlBu' )
plt.colorbar()
plt.title('Shallow Groundwater Percentile 26.10.20')